# Imports

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

In [2]:
# Read the dataset
cleaned_df = pd.read_csv('/Users/priyakundu/Documents/NYU Capstone WaterVue Files/Data Processing/Updated_Dataframe_WaterQual.csv')
cleaned_df

,Location,Sample Date,Chlorophyll A,Dissolved Oxygen,Salinity,Specific Conductance,Total Nitrogen,Total Phosphorus,Turbidity
0,#1 HILLSBORO CANAL US 1,2006-02-28,12.100000,6.980000,19.400000,10.351373,0.830000,0.086000,2.400000
1,#1 HILLSBORO CANAL US 1,2006-08-31,4.430000,5.540000,15.750000,10.165844,0.981000,0.109000,1.400000
2,#1 HILLSBORO CANAL US 1,2007-02-28,2.605000,6.195000,23.200000,10.501905,0.754000,0.083500,1.650000
3,#1 HILLSBORO CANAL US 1,2007-08-31,4.890000,4.730000,31.100000,10.774781,0.777000,0.094000,2.300000
4,#1 HILLSBORO CANAL US 1,2008-02-29,5.925000,5.770000,12.500000,9.893361,1.440000,0.102000,2.050000
...,...,...,...,...,...,...,...,...,...
853,#89 NOB HILL RD POMPANO CANAL,2020-02-29,3.933333,5.703333,0.310000,6.451930,1.097133,0.014000,0.750000
854,#89 NOB HILL RD POMPANO CANAL,2020-08-31,1.570000,7.760000,0.250000,6.265301,0.910000,0.011000,0.000000
855,#89 NOB HILL RD POMPANO CANAL,2021-02-28,2.966667,5.833333,0.233333,6.153383,0.984033,0.003667,0.566667
856,#89 NOB HILL RD POMPANO CANAL,2021-08-31,5.510000,5.370000,0.270000,6.324359,1.410500,0.049000,0.775000


## Splitting

In [3]:
sorted_df = cleaned_df.sort_values(by='Sample Date', ascending=True)

In [4]:
# Split data into train and test sets
train_data = sorted_df.iloc[:-3*cleaned_df["Location"].nunique()].sort_values(by=['Location', 'Sample Date'], ascending=True)  # Use all but the last 12 months for training
test_data = sorted_df.iloc[-3*cleaned_df["Location"].nunique():].sort_values(by=['Location', 'Sample Date'], ascending=True)   # Use the last 12 months for testing

# Forecasting

In [5]:
attributes = ['Chlorophyll A', 'Dissolved Oxygen', 'Salinity', 'Specific Conductance', 'Total Nitrogen', 'Total Phosphorus', 'Turbidity']

In [6]:
final_dict = {}
sequence_length = 3  # Example sequence length for LSTM model
hyper_params = [(64, 'relu'), (128, 'relu'), (64, 'tanh')]  # Example hyperparameters for LSTM model

for location in train_data["Location"].unique():
    att_rmse = {}
    train_df = train_data[train_data["Location"] == location].set_index("Sample Date")
    train_df.index = pd.to_datetime(train_df.index)  # Convert index to DateTimeIndex
    test_df = test_data[test_data["Location"] == location]

    for attribute in attributes:
        rmses = []
        for hyper in hyper_params:
            # Normalize data
            scaler = MinMaxScaler()
            train_scaled = scaler.fit_transform(train_df[[attribute]]) 
            
            # Prepare data for LSTM model
            train_scaled = scaler.fit_transform(train_df[attribute].values.reshape(-1, 1))
            train_generator = TimeseriesGenerator(train_scaled, train_scaled, length=sequence_length, batch_size=1)
            
            # Define and compile LSTM model
            model = Sequential([
                LSTM(hyper[0], activation=hyper[1], input_shape=(sequence_length, 1)),
                Dense(1)
            ])
            model.compile(optimizer='adam', loss='mean_squared_error')
            
            # Fit the model
            model.fit(train_generator, epochs=20, verbose=0)
            
            # Prepare last sequence for forecasting
            last_sequence = train_scaled[-sequence_length:]
            
            # Iteratively forecast the next 3 steps
            forecast_steps = 3
            predictions_scaled = []
            for _ in range(forecast_steps):
                # Reshape the last sequence for prediction
                last_sequence_reshaped = last_sequence.reshape((1, sequence_length, 1))
                # Predict the next step and append to predictions
                next_step_pred = model.predict(last_sequence_reshaped)
                predictions_scaled.append(next_step_pred.ravel()[0])
                # Update the last sequence with the prediction
                last_sequence = np.roll(last_sequence, -1)
                last_sequence[-1] = next_step_pred
            
            # Inverse transform predictions
            predictions_inv = scaler.inverse_transform(np.array(predictions_scaled).reshape(-1, 1))
            
            # Compute RMSE for each attribute
            mse = mean_squared_error(test_df[attribute].values[:forecast_steps], predictions_inv)
            rmse = np.sqrt(mse)
            rmses.append(rmse)
        
        att_rmse[attribute] = rmses
    
    # Initialize MinMaxScaler
    scaler = MinMaxScaler()
    
    att_rmse_df = pd.DataFrame(att_rmse)
    att_rmse_df[attributes] = scaler.fit_transform(att_rmse_df[attributes].values)
    final_dict[location] = hyper_params[att_rmse_df.sum(axis=1).argmin()]

final_dict_df = pd.DataFrame({"Locations": final_dict.keys(), "Hyper Parameters": final_dict.values()})

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


In [7]:
final_dict_df = pd.DataFrame({"Locations": final_dict.keys(), "Hyper Parameters": final_dict.values()})

In [8]:
# Print the final results
final_dict_df

,Locations,Hyper Parameters
0,#1 HILLSBORO CANAL US 1,"(64, relu)"
1,#10 MIDDLE RIVER E SUNRISE,"(128, relu)"
2,#110 POMPANO CANAL AT DIXIE AN,"(128, relu)"
3,#111 S. FORK MID R. @ N.E. 15,"(64, relu)"
4,#12 MIDDLE RIVER NW 31ST AVE,"(64, tanh)"
5,#15 NEW RIVER ANDREWS AVE,"(64, relu)"
6,#17 PLANTATION CANAL @ S-33,"(64, tanh)"
7,#19 NEW RIVER RIVER REACH,"(64, tanh)"
8,#22 N NEW RIVER SW 125 AVE,"(64, relu)"
9,#24 DANIA CUT-OFF US-1,"(64, tanh)"


In [9]:
att_rmse = {attribute: [] for attribute in attributes}

for location in final_dict:
    hyper = final_dict[location]
    train_df = train_data[train_data["Location"] == location].set_index("Sample Date")
    train_df.index = pd.to_datetime(train_df.index)  # Convert index to DateTimeIndex
    test_df = test_data[test_data["Location"]==location]

    for attribute in attributes:

        # Normalize data
        scaler = MinMaxScaler()
        train_scaled = scaler.fit_transform(train_df[[attribute]]) 

        # Prepare data for LSTM model
        train_scaled = scaler.fit_transform(train_df[attribute].values.reshape(-1, 1))
        train_generator = TimeseriesGenerator(train_scaled, train_scaled, length=sequence_length, batch_size=1)

        # Define and compile LSTM model
        model = Sequential([
            LSTM(hyper[0], activation=hyper[1], input_shape=(sequence_length, 1)),
            Dense(1)
        ])
        model.compile(optimizer='adam', loss='mean_squared_error')

        # Fit the model
        model.fit(train_generator, epochs=20, verbose=0)

        # Prepare last sequence for forecasting
        last_sequence = train_scaled[-sequence_length:]

        # Iteratively forecast the next 3 steps
        forecast_steps = 3
        predictions_scaled = []
        for _ in range(forecast_steps):
            # Reshape the last sequence for prediction
            last_sequence_reshaped = last_sequence.reshape((1, sequence_length, 1))
            # Predict the next step and append to predictions
            next_step_pred = model.predict(last_sequence_reshaped)
            predictions_scaled.append(next_step_pred.ravel()[0])
            # Update the last sequence with the prediction
            last_sequence = np.roll(last_sequence, -1)
            last_sequence[-1] = next_step_pred

        # Inverse transform predictions
        predictions_inv = scaler.inverse_transform(np.array(predictions_scaled).reshape(-1, 1))

        # Compute RMSE for each attribute
        mse = mean_squared_error(test_df[attribute].values[:forecast_steps], predictions_inv)
        rmse = np.sqrt(mse)
        att_rmse[attribute].append(rmse)

final_dict_df = pd.concat([final_dict_df, pd.DataFrame(att_rmse)], axis=1)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


In [10]:
final_dict_df

,Locations,Hyper Parameters,Chlorophyll A,Dissolved Oxygen,Salinity,Specific Conductance,Total Nitrogen,Total Phosphorus,Turbidity
0,#1 HILLSBORO CANAL US 1,"(64, relu)",1.293565,0.194153,5.034117,0.729773,0.026227,0.011863,0.379120
1,#10 MIDDLE RIVER E SUNRISE,"(128, relu)",1.775179,0.159865,3.798971,0.160301,0.215166,0.057554,0.448139
2,#110 POMPANO CANAL AT DIXIE AN,"(128, relu)",2.511119,0.783283,0.022176,0.123192,0.113269,0.008487,0.630513
3,#111 S. FORK MID R. @ N.E. 15,"(64, relu)",2.867971,0.690026,7.933667,1.404722,0.228061,0.018143,0.224704
4,#12 MIDDLE RIVER NW 31ST AVE,"(64, tanh)",0.750188,1.036168,0.026033,0.110162,0.276793,0.006993,0.227370
5,#15 NEW RIVER ANDREWS AVE,"(64, relu)",1.974876,0.865889,10.584856,1.102806,0.417160,0.041882,0.397299
6,#17 PLANTATION CANAL @ S-33,"(64, tanh)",2.454606,2.185787,2.742909,0.668451,0.253256,0.037178,0.301443
7,#19 NEW RIVER RIVER REACH,"(64, tanh)",2.329613,0.853731,4.162968,1.075410,0.240823,0.018251,0.413071
8,#22 N NEW RIVER SW 125 AVE,"(64, relu)",1.279221,1.392459,0.025584,0.074676,0.189482,0.012631,0.356083
9,#24 DANIA CUT-OFF US-1,"(64, tanh)",0.783559,0.690378,10.997656,0.601193,0.267216,0.041201,0.129473


In [11]:
accuracy = 1 - (final_dict_df.iloc[:,2:]/final_dict_df.iloc[:,2:].max())

accuracy.insert(0,"Locations", final_dict_df["Locations"])
accuracy.insert(1,"Hyper Parameters", final_dict_df["Hyper Parameters"])

accuracy

,Locations,Hyper Parameters,Chlorophyll A,Dissolved Oxygen,Salinity,Specific Conductance,Total Nitrogen,Total Phosphorus,Turbidity
0,#1 HILLSBORO CANAL US 1,"(64, relu)",0.746131,0.911175,0.542255,0.480485,0.937129,0.879653,0.570194
1,#10 MIDDLE RIVER E SUNRISE,"(128, relu)",0.651612,0.926862,0.654565,0.885884,0.484212,0.416116,0.491947
2,#110 POMPANO CANAL AT DIXIE AN,"(128, relu)",0.507180,0.641647,0.997984,0.912302,0.728476,0.913899,0.285190
3,#111 S. FORK MID R. @ N.E. 15,"(64, relu)",0.437146,0.684312,0.278604,0.000000,0.453301,0.815937,0.745254
4,#12 MIDDLE RIVER NW 31ST AVE,"(64, tanh)",0.852772,0.525952,0.997633,0.921578,0.336482,0.929061,0.742232
5,#15 NEW RIVER ANDREWS AVE,"(64, relu)",0.612421,0.603855,0.037535,0.214929,0.000000,0.575114,0.549584
6,#17 PLANTATION CANAL @ S-33,"(64, tanh)",0.518271,0.000000,0.750592,0.524140,0.392903,0.622836,0.658256
7,#19 NEW RIVER RIVER REACH,"(64, tanh)",0.542802,0.609417,0.621468,0.234432,0.422708,0.814842,0.531703
8,#22 N NEW RIVER SW 125 AVE,"(64, relu)",0.748946,0.362948,0.997674,0.946839,0.545782,0.871864,0.596310
9,#24 DANIA CUT-OFF US-1,"(64, tanh)",0.846223,0.684151,0.000000,0.572020,0.359440,0.582018,0.853217


In [12]:
accuracy.describe()

,Chlorophyll A,Dissolved Oxygen,Salinity,Specific Conductance,Total Nitrogen,Total Phosphorus,Turbidity
count,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000
mean,0.650590,0.634889,0.714492,0.713805,0.573589,0.701218,0.554642
std,0.212113,0.244035,0.313801,0.281317,0.242405,0.237101,0.216833
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.555077,0.527831,0.562059,0.560099,0.430356,0.576840,0.501886
50%,0.637326,0.638557,0.762268,0.862978,0.554810,0.802910,0.601567
75%,0.797097,0.784565,0.995148,0.911634,0.782972,0.886987,0.704863
max,0.981959,0.983022,0.998582,0.992651,0.937129,0.940530,0.859026
